<a href="https://colab.research.google.com/github/AIAlchemy1/Generative-AI/blob/main/02_LangChain/LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The world of LLM and LLM related tools changes very fast.

For example, while I was writing this code, one of most widely used libraries for LLM-related tasks deprecated their main interface.

More precisely they went from `Chain` interface to `LCEL` - Lang Chain Expression Language.

This change does not change the fundamental materials so much. But we wanted to show you the difference between the two, as you might see case studies in LCEL and I want you to be prepared

In [1]:
!pip install langchain openai langchain_openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requi

In [2]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.environ['OPENAI_API_KEY'] = open("/content/drive/MyDrive/.open-ai-api-key.txt").read().strip()

Mounted at /content/drive


A short demonstration of how the new interface looks:

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(chain.invoke({"topic": "beavers and elves"}))

Why don't beavers work with elves?

Because they can't stand the elf and safety regulations!


This notebook will have a demonstration of all the concepts introduced in the previous code and how (or if) they changed with the new version.

# Using a model

Before we were using a method `predict` and now the correct one is `invoke`

## Classic LLM

In [4]:
from langchain.llms import OpenAI

llm = OpenAI()

llm.invoke("Hello ")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


'\n\nHello, how can I assist you?'

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("One {object} a day keeps who away?")

prompt_value = prompt.invoke({"object": "apple"})

llm.invoke(prompt_value.to_string())

'\n\nDoctor\n'

## Chat model

In [6]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

chat_model.invoke(prompt_value.to_messages())

AIMessage(content='The saying goes "An apple a day keeps the doctor away."')

# Chains

What it looked like before

```
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("Australia"
```

Now the interface is the following

In [7]:
chain = prompt | model
chain.invoke({"object": "Banana"})

AIMessage(content='There isn\'t a specific person or thing that a banana a day keeps away. This might be a misinterpretation of the saying "An apple a day keeps the doctor away." Eating a banana a day can contribute to a healthy diet due to its high potassium and vitamin C content.')

## Sequential chains

Before it looked like this:

```
from langchain.chains import SimpleSequentialChain

first_prompt = PromptTemplate.from_template(
    "What is the capital of {country}?"
)
first_chain = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = PromptTemplate.from_template(
    "{city} is the capital of which country?"
)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

simple_sequential_chain = SimpleSequentialChain(
    chains=[first_chain, second_chain],
    verbose=True
)
```
And the new interface is the following:


In [8]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

output_parser = StrOutputParser()

pie_prompt = ChatPromptTemplate.from_template("I want to bake a {pie}. Give me a list of ingredients.")

looking_in_the_fridge_prompt = ChatPromptTemplate.from_template(
    "I have {in_the_fridge}, repeat {recipe} adding to each ingredient if I need to buy it"
)

model = ChatOpenAI()

recipe_chain = pie_prompt | model | output_parser

ingredients_chain = (
   {"in_the_fridge": itemgetter("in_the_fridge"), "recipe": recipe_chain}
   | looking_in_the_fridge_prompt
   | model
   | output_parser
)

print(
    ingredients_chain.invoke(
        {"pie": "cheescake", "in_the_fridge": "milk"},
    )
)

You will need to buy the following ingredients:

- Graham cracker crumbs
- Unsalted butter
- Granulated sugar
- Cream cheese
- Eggs
- Vanilla extract
- Sour cream
- Fresh berries or fruit compote (if desired)
- Springform pan (if you don't already have one)


## Debugging

In [9]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

print(
    ingredients_chain.invoke(
        {"pie": "cheescake", "in_the_fridge": "milk"},
        config={'callbacks': [ConsoleCallbackHandler()]}
    )
)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "pie": "cheescake",
  "in_the_fridge": "milk"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<in_the_fridge,recipe>] Entering Chain run with input:
{
  "pie": "cheescake",
  "in_the_fridge": "milk"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<in_the_fridge,recipe> > 3:chain:RunnableLambda] Entering Chain run with input:
{
  "pie": "cheescake",
  "in_the_fridge": "milk"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<in_the_fridge,recipe> > 3:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "milk"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<in_the_fridge,recipe> > 4:chain:RunnableSequence] Entering Chain run with input:
{
  "pie": "cheescake",
  "in_the_fridge": "milk"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<in_the_fridge,recipe> > 4:chain:RunnableSequence > 5:prompt:ChatProm

# Using tools

Before it looked like this:

```
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchRun

llm = OpenAI(temperature=0)

tools = [DuckDuckGoSearchRun()]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
```

In [10]:
!pip install --upgrade --quiet  langchain langchain-openai duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 32.7 MB/s eta 0:00:00


In the new version it looks like this:


In [11]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

search = DuckDuckGoSearchRun()

template = """turn the following user input into a search query for a search engine:

{input}"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = prompt | model | StrOutputParser() | search

chain.invoke({"input": "What are the most famous K-pop bands?"})

"This group took the world by storm which make them the most famous k-pop bands in the world. Their sense of music, their up-beats, choreography of each and every music album makes a den in heart of their fans. This group started in June 12, 2013 with seven members named RM, Jin, Suga, J-Hope, Jimin, V, and Jungkook. Since then they are unstoppable. The band comprising Jennie Kim, Lisa, Jisoo, and Rose is one of the most famous K-pop bands, and not just that, the members are equally successful individually. BIGBANG: One of the most famous K-pop bands formed by YG Entertainment. EXO: A famous South Korean K-pop group. Jungkook: A famous singer best known for his work in the boy band group, BTS. Moon Jae-in: The 12 th and current President of South Korea. Onew: A well-known K-Pop singer and actor. BTS is one of the most famous K-pop bands worldwide. The South Korean boy band debuted in 2013. Known BTS members are Jungkook, Jimin, Jin, RM, V, J-Hope, and Suga. They sing, rap, and dance. T

# Using memory

Before

```
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain


prefix = """Have a conversation with a human, answering the following """\
    """questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    verbose=True
)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)
```

In [12]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [13]:
inputs = {"input": "I want to get a cat, do you think it's a good idea?"}
response = chain.invoke(inputs)
response

AIMessage(content="As an AI, I don't have personal opinions, but I can provide you with some information to help you make your decision. Getting a cat can be a wonderful and rewarding experience for many people. Cats can provide companionship, reduce stress, and bring joy to your life. However, it's important to consider a few factors before getting a cat. \n\n1. Time and Commitment: Cats require time and attention. They need to be fed, groomed, and played with regularly. Make sure you have enough time to devote to your cat's needs.\n\n2. Financial Responsibility: Owning a cat comes with expenses such as food, litter, veterinary care, and toys. Consider if you are financially prepared to provide for a cat's needs.\n\n3. Allergies: If you or someone in your household is allergic to cats, it may not be the best idea to get one unless you're willing to explore hypoallergenic breeds or take other measures to manage allergies.\n\n4. Living Situation: Ensure that your living situation allows

In [14]:
memory.save_context(inputs, {"output": response.content})

In [15]:
inputs = {"input": "I want it, what would be a good name for it?"}
response = chain.invoke(inputs)
response

AIMessage(content="Choosing a name for your cat can be a fun and personal decision. Here are a few suggestions, but remember, the best name is the one that feels right for you and your cat:\n\n1. Whiskers\n2. Luna\n3. Oliver\n4. Bella\n5. Charlie\n6. Simba\n7. Lily\n8. Max\n9. Chloe\n10. Leo\n\nConsider your cat's personality, appearance, or even your favorite book, movie, or TV show for inspiration. Take your time, and you'll find the perfect name that suits your new furry friend!")